In [3]:
#!pip install undetected-chromedriver
#!pip install selenium
#!pip install webdriver_manager

In [4]:
#from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.common.action_chains import ActionChains
#from selenium_stealth import stealth


#from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from pprint import pprint, pformat
import requests
import os

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import random
from pathlib import Path
import shutil
from urllib.parse import unquote

from concurrent.futures import ThreadPoolExecutor, as_completed
#from PIL import Image
#import io

In [5]:

class PasrerYandexRu():
    def __init__(self, search_string, max_pict_count=1000, debug_log=False):
        # требуемое количество картинок
        self._max_pict_count = max_pict_count
        # выводить или нет отладочную информацию
        self._debug_log = debug_log
        # искомая строка
        self._search_string = search_string
        # ссылки на найденные картинки
        self._hrefs = []
        self._parsed_cells = []
    
    @staticmethod
    def pause(min_second=1, max_second=5):
        """Пауза случайной длины в заданных пределах"""
        time.sleep(random.randint(min_second, max_second))

    @property
    def search_string(self):
        """Искомая строка"""
        return self._search_string
    
    @property
    def root_folder(self):
        """Корневая папка для сохранения изображений"""
        return self._root_folder
    
    @property
    def result_folder(self):
        """Папка для сохранения результатов конкретного поиска"""
        return self.path_result_folder
    
    @property
    def hrefs(self):
        """Ссылки на найденные картинки"""
        return self._hrefs
        
    @staticmethod
    def extract_picture_url(source_url):
        """Извлечь url картинки из url из результатов поиска"""
        unquoted_url = unquote(source_url)
        unquoted_url = unquoted_url[unquoted_url.find('&img_url=')+9:unquoted_url.find('&text=')]
        return unquoted_url
    
    def build_seach_url(self):
        """Построить url для поиска картинок"""
        return f'https://yandex.ru/images/search?text={self._search_string}'
    
    def get_links(self):
        #with uc.Chrome(version_main=132,service=ChromiumService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())) as browser:
        try:
            browser = uc.Chrome(version_main=132,service=ChromiumService(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()))
            browser.implicitly_wait(10)
            # открыть в браузере результаты поиска
            browser.get(self.build_seach_url())
            PasrerYandexRu.pause()
            html = browser.find_element(By.TAG_NAME, 'html')
            prev_cells_count = 0
            cells = []
            # сначала проскроллим страницу вниз
            while len(cells) < int(self._max_pict_count * 1.5):
                cells = browser.find_elements(By.CLASS_NAME, 'SerpItem-Thumb')
                print(f'len(cells)={len(cells)}')
                # нажать кнопку End, чтобы подгрузились еще картинки
                html.send_keys(Keys.END)
                print('Нажали кнопоку End')
                PasrerYandexRu.pause()

                if prev_cells_count == len(cells):
                    # если на странице есть кнопка "показать еще" - нажать ее
                    button_next = browser.find_elements(By.CLASS_NAME, "FetchListButton-Button")
                    if button_next is not None and len(button_next) > 0:
                        PasrerYandexRu.pause()
                        button_next[0].click()
                        print('Нажали кнопку Показать еще')
                    else:
                        print("Дошли до конца?")
                        PasrerYandexRu.pause(10, 20)
                        break
                prev_cells_count = len(cells)

            PasrerYandexRu.pause()
            # пройдемся по "ячейкам" с картинками и вытащим ссылки на картинки
            cells = browser.find_elements(By.CLASS_NAME, 'SerpItem-Thumb')
            for cell in cells:
                ImagesContentImage_Cover_elements = cell.find_elements(By.CLASS_NAME, 'ImagesContentImage-Cover')
                print(f'ImagesContentImage_Cover_elements={len(ImagesContentImage_Cover_elements)}')
                # выделим в ячейке с картинками кликабельную ссылку, чтобы открыть картинку в оригинальном разрешении
                image_info = ImagesContentImage_Cover_elements[0].find_elements(By.CLASS_NAME, 'ImagesContentImage-Image_clickable')[0]
                cell_href = image_info.get_attribute('src')

                print(f'{len(self._parsed_cells):05}: {cell_href} ({image_info.get_attribute('alt')})')
                if cell_href not in self._parsed_cells:
                    print('added...')
                    self._parsed_cells.append(cell_href)
                    browser.execute_script("arguments[0].click();", image_info)
                    #image_info.click()
                    PasrerYandexRu.pause(1,3)
                    href = browser.find_elements(By.CLASS_NAME, 'MMImage-Origin')[0].get_attribute('src')
                    print(f'append image href={href}')
                    if href not in self._hrefs:
                        self._hrefs.append(href)
                    if len(self._hrefs) >= self._max_pict_count:
                        print(f'RETURN!!!! {len(self._hrefs)}, {self._max_pict_count}')
                        return
                    browser.find_elements(By.CLASS_NAME, 'ImagesViewer-Close')[0].click()
                    print('close image viewer')
                    PasrerYandexRu.pause(1,3)
                else:
                    print(f'skipped {cell_href}')
            print('Страница обработана')
            
        except Exception as e:
            print('except')
            print(e)
        finally:
            print('finally')
            browser.close()
            browser.quit()

    def main(self):
        if self._debug_log:
            print(f"Строка поиска: {self._search_string}")
        self.get_links()
        if self._debug_log:
            print(f"Найдено ссылок: {len(self._hrefs)}")

In [6]:
class ImagesDownloader():
    
    def __init__(self, hrefs, folder):
        self._hrefs = hrefs
        self._folder = folder
        # корневой каталог для сохранения результатов поиска
        self._root_folder = Path(Path.cwd(),'dataset')
        # каталог для сохранения результатов конкретного поиска
        self._path_result_folder = Path(self._root_folder, self._folder)
        # создать словарь ИмяФайла: ссылка
        self._hrefs_dict = {}
        for i in range(len(self._hrefs)):
            file_name = Path(self._path_result_folder, f'{i:05}')
            self._hrefs_dict[file_name] = self._hrefs[i]
        print(len(self._hrefs_dict.keys()))

    def prepare_folder(self):
        """Подготовить каталоги для сохранения картинок: если нет - создать, если есть - удалить со всем содержимым и создать заново"""
        if os.path.exists(self._path_result_folder) and os.path.isdir(self._path_result_folder):
            shutil.rmtree(self._path_result_folder)
        print(self._path_result_folder)
        os.makedirs(self._path_result_folder, exist_ok=True)
            
    @staticmethod
    def download_and_save_image(url, filename):
        """Скачать картинку по ссылке и сохранить ее в файл"""
        res = requests.get(url)
        if res.status_code != 200:
            print(url, res.status_code)
        else:
            print(url, len(res.content))
        #img = Image.open(io.BytesIO(res.content))
        #img.save(f'{filename}.{img.format}')
        with open(f'{filename}.jpeg', 'wb') as f:
            f.write(res.content)

    def download_images(self):
        """Скачать картинки в один поток"""
        self.prepare_folder()
        for filename, url in self._hrefs_dict.items():
            ImagesDownloader.download_and_save_image(url, filename)

    def download_images_parallel(self):
        """Скачать картинки многопоточно"""
        self.prepare_folder()
        with ThreadPoolExecutor(max_workers=5) as executor:
            # Запускаем задачи
            futures = [executor.submit(ImagesDownloader.download_and_save_image, url, filename) for filename, url in self._hrefs_dict.items()]

In [7]:
parse1 = PasrerYandexRu(search_string="polar bear", debug_log=True)
#parse1 = PasrerYandexRu(search_string="fox", max_pict_count=10, debug_log=True)
parse1.main()

Строка поиска: polar bear
len(cells)=85
Нажали кнопоку End
len(cells)=140
Нажали кнопоку End
len(cells)=195
Нажали кнопоку End
len(cells)=250
Нажали кнопоку End
len(cells)=305
Нажали кнопоку End
len(cells)=360
Нажали кнопоку End
len(cells)=415
Нажали кнопоку End
len(cells)=470
Нажали кнопоку End
len(cells)=525
Нажали кнопоку End
len(cells)=525
Нажали кнопоку End
Нажали кнопку Показать еще
len(cells)=580
Нажали кнопоку End
len(cells)=635
Нажали кнопоку End
len(cells)=690
Нажали кнопоку End
len(cells)=745
Нажали кнопоку End
len(cells)=800
Нажали кнопоку End
len(cells)=855
Нажали кнопоку End
len(cells)=910
Нажали кнопоку End
len(cells)=965
Нажали кнопоку End
len(cells)=1020
Нажали кнопоку End
len(cells)=1075
Нажали кнопоку End
len(cells)=1075
Нажали кнопоку End
Нажали кнопку Показать еще
len(cells)=1130
Нажали кнопоку End
len(cells)=1181
Нажали кнопоку End
len(cells)=1233
Нажали кнопоку End
len(cells)=1286
Нажали кнопоку End
len(cells)=1340
Нажали кнопоку End
len(cells)=1394
Нажали кнопок

In [8]:
downloader = ImagesDownloader(hrefs=parse1._hrefs, folder=parse1.search_string)
downloader.download_images_parallel()
#downloader.download_images()

1000
/home/igel/Projects/ml/ml-inno-hw/2.Python/2.12.1 Web scrapping/dataset/polar bear
https://avatars.mds.yandex.net/i?id=1200976d66152775aeeb4213dc144802_l-7047516-images-thumbs&n=13 114487
https://avatars.mds.yandex.net/i?id=3a81a81a6666bbeb6d9df4a7206ea396-5234764-images-thumbs&n=13 171003
https://avatars.mds.yandex.net/i?id=fc959bb293a8132a7931c436b84d7165_l-10310276-images-thumbs&n=13 124175
https://avatars.mds.yandex.net/i?id=d3960e003c1a24ec610d19a3a3678cd9_l-8750921-images-thumbs&n=13 123615
https://avatars.mds.yandex.net/i?id=f23948a50b17dce833fdc8b39ad6f013_l-9029424-images-thumbs&n=13 297527
https://avatars.mds.yandex.net/i?id=cba78f2abf26b13e217e4911f7362c81_l-4755642-images-thumbs&n=13 22871
https://avatars.mds.yandex.net/i?id=6e06e7537213b7e9f1ef388642e8bf9c_l-7753970-images-thumbs&n=13 28239
https://avatars.mds.yandex.net/i?id=150f31450941af214e458d53526964a3_l-13457071-images-thumbs&n=13 82223
https://avatars.mds.yandex.net/i?id=d29fb2e7ae1a1bacd3c1742a2cebae86_l-9181

In [9]:
parse2 = PasrerYandexRu(search_string="brown bear", debug_log=True)
#parse2 = PasrerYandexRu(search_string="red panda", max_pict_count=30, debug_log=True)
parse2.main()

Строка поиска: brown bear
len(cells)=85
Нажали кнопоку End
len(cells)=140
Нажали кнопоку End
len(cells)=195
Нажали кнопоку End
len(cells)=250
Нажали кнопоку End
len(cells)=305
Нажали кнопоку End
len(cells)=360
Нажали кнопоку End
len(cells)=415
Нажали кнопоку End
len(cells)=470
Нажали кнопоку End
len(cells)=525
Нажали кнопоку End
len(cells)=525
Нажали кнопоку End
Нажали кнопку Показать еще
len(cells)=580
Нажали кнопоку End
len(cells)=635
Нажали кнопоку End
len(cells)=690
Нажали кнопоку End
len(cells)=745
Нажали кнопоку End
len(cells)=800
Нажали кнопоку End
len(cells)=855
Нажали кнопоку End
len(cells)=910
Нажали кнопоку End
len(cells)=965
Нажали кнопоку End
len(cells)=1020
Нажали кнопоку End
len(cells)=1075
Нажали кнопоку End
len(cells)=1075
Нажали кнопоку End
Нажали кнопку Показать еще
len(cells)=1130
Нажали кнопоку End
len(cells)=1184
Нажали кнопоку End
len(cells)=1239
Нажали кнопоку End
len(cells)=1294
Нажали кнопоку End
len(cells)=1349
Нажали кнопоку End
len(cells)=1395
Нажали кнопок

In [10]:
downloader = ImagesDownloader(hrefs=parse2._hrefs, folder=parse2.search_string)
downloader.download_images_parallel()

1000
/home/igel/Projects/ml/ml-inno-hw/2.Python/2.12.1 Web scrapping/dataset/brown bear
https://i.pinimg.com/originals/8b/5e/be/8b5ebe12c3a61802dfdaaf4df393dda7.png 813486
https://avatars.mds.yandex.net/i?id=c778033e1a2ea67eed3e1771f0aee5fc_l-12500642-images-thumbs&n=13 191183
https://avatars.mds.yandex.net/i?id=4be6a49f32536b524ae84fca95c21984_l-5660190-images-thumbs&n=13 164114
https://avatars.mds.yandex.net/i?id=99dd2b9349c40e763171876d156295c8_l-4967643-images-thumbs&n=13 194167
https://avatars.mds.yandex.net/i?id=7dd55d82223f6ba94c0b9dbff09f1faf_l-9873353-images-thumbs&n=13 73324
https://avatars.mds.yandex.net/i?id=127b2723ff690cb7797579b26e9cbf46_l-10695517-images-thumbs&n=13 1044433
https://avatars.mds.yandex.net/i?id=fd3efcfe01ecbdf0da2803cf8268a260_l-9107657-images-thumbs&n=13 344990
https://avatars.mds.yandex.net/i?id=129d6a6a99e88b1d78923f5b4338f715_l-5231013-images-thumbs&n=13 164711
https://avatars.mds.yandex.net/i?id=67db96a3aac1fc579acedb89760dbf51_l-10993330-images-thum